In [ ]:
# Install PyTorch Geometric (versi yang stabil di Colab)
!pip install torch==2.1.0 torchvision torchaudio
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
!pip install torch-geometric




INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.

In [5]:
!pip install numpy==1.26.4 --force-reinstall
!pip install pandas numpy torch
!pip install scikit-learn

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [6]:
import pandas as pd
import numpy as np
import torch
import ast
from sklearn.decomposition import PCA
from torch_geometric.data import Data
import joblib

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# --- 1. BACA CSV ---
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Hasil_Ekstraksi/graph_dataset_with_color_moment_fix.csv")


In [9]:
# --- 2. Parse Node Features ---
import ast
import numpy as np

subset = df["Node_Features"].iloc[:10]  # hanya ambil 50 baris
all_features_sample = np.array([ast.literal_eval(s) for s in subset])
print(all_features_sample.shape)


(10, 9, 3209)


In [13]:
# --- 3. PCA ---
from sklearn.decomposition import PCA

# Ambil dimensi dari array
num_graphs, num_nodes, feat_dim = all_features_sample.shape

# Flatten dari 3D → 2D: [num_graphs * num_nodes, feat_dim]
flat = all_features_sample.reshape(-1, feat_dim)

# Apply PCA
pca = PCA(n_components=90)
flat_reduced = pca.fit_transform(flat)

# Reshape kembali ke 3D: [num_graphs, num_nodes, 512]
reduced = flat_reduced.reshape(num_graphs, num_nodes, -1)

print(reduced.shape)


(10, 9, 90)


In [15]:
# --- 4. Build Graph DataList ---
data_list = []

for i in range(reduced.shape[0]):
    row = df.iloc[i]
    x = torch.tensor(reduced[i], dtype=torch.float)

    edge_index = torch.tensor(ast.literal_eval(row["Edge_Index"]), dtype=torch.long).t().contiguous()
    y = torch.tensor(0 if row["Category"] == "tidak_estetik" else 1, dtype=torch.long)

    data = Data(x=x, edge_index=edge_index, y=y)
    data.file_name = row["Filename"]
    data_list.append(data)

# --- 5. Save to .pt ---
torch.save(data_list, "/content/drive/MyDrive/Hasil_Ekstraksi/processed_graphs_pca.pt")

# --- 6. Simpan model PCA juga jika perlu ---
joblib.dump(pca, "/content/drive/MyDrive/Hasil_Ekstraksi/pca_model_512.joblib")

print(f"Sukses simpan {len(data_list)} graf ke .pt!")

Sukses simpan 10 graf ke .pt!


In [17]:
import torch

# Load file
loaded_data = torch.load("/content/drive/MyDrive/Hasil_Ekstraksi/processed_graphs_pca.pt")

# Lihat contoh 1 graph
print(loaded_data[0])
data = data_list[0]       # Ambil 1 graph
print(data.x)             # Tampilkan node features-nya
print(data.x.shape)       # (jumlah_node, dimensi_fitur)



Data(x=[9, 90], edge_index=[12, 2], y=0, file_name='19990.jpg')
tensor([[ 7.1698e+01,  2.4917e+01, -2.2921e+01,  1.2838e+00, -8.2267e+00,
          4.1700e+00,  2.4401e+00, -4.0891e+00,  1.7803e+00,  5.5417e+00,
          2.6762e+00, -6.8683e-01,  6.4655e-01,  1.8303e+00, -4.8176e-01,
          7.3803e-01,  1.1373e+00, -1.8151e+00,  1.8923e+00,  4.5947e-01,
         -1.5340e+00, -3.0310e-01, -1.8287e+00,  9.5483e-01, -5.0072e-01,
          1.0203e+00,  1.2441e+00, -2.6882e+00,  1.6940e+00,  2.1146e-02,
         -5.5187e-02,  2.0713e+00, -5.6154e-01,  1.3749e-01, -4.2177e-02,
          1.1316e+00, -2.1871e-01,  5.8719e-01, -4.5432e-01,  2.1926e-02,
          1.5011e-01,  8.6480e-01,  6.0046e-01,  2.7685e-01,  4.1994e-01,
          5.5074e-02,  7.0737e-01, -1.3263e-01, -2.5641e-01, -1.7694e-01,
         -1.5122e-01, -3.0872e-02, -4.6369e-01, -3.4872e-01,  8.3397e-02,
          3.2720e-01,  9.2747e-02,  7.4644e-01,  1.8297e-02,  4.7919e-02,
          1.9401e-01,  3.6249e-01, -4.3518e-01, 

In [ ]:
import ast
import numpy as np

batch_size = 100
total = len(df)

for start in range(0, total, batch_size):
    end = min(start + batch_size, total)
    print(f"🔄 Processing batch {start} to {end - 1}...")

    # Ambil subset batch
    subset = df["Node_Features"].iloc[start:end]

    # Parse list of string → list of list
    batch_features = np.array([ast.literal_eval(s) for s in subset])

    # Simpan hasil batch
    np.save(f"/content/drive/MyDrive/batch_features_{start}_{end-1}.npy", batch_features)

    print(f"✅ Saved batch {start}–{end - 1} | shape: {batch_features.shape}")


In [ ]:
import numpy as np
import glob

# Ambil semua file .npy yang tersimpan
batch_files = sorted(glob.glob("/content/drive/MyDrive/batch_features_*.npy"))

# Load dan gabungkan semuanya
all_features = np.concatenate([np.load(f) for f in batch_files])

print("✅ Final shape:", all_features.shape)
